# FinRL Data Wrangle

## Install and Import Packages

In [ ]:
%pip install swig
%pip install wrds
%pip install pyportfolioopt
%pip install yfinance
%pip install box2d-py

In [ ]:
# finrl
%pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

In [23]:
import os
import pandas as pd
import numpy as np
import datetime
import yfinance as yf

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl import config_tickers
from finrl.config import INDICATORS

import itertools

In [7]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2023-05-01'

In [8]:
symbols = [
    'aapl',
    'msft',
    'meta',
    'ibm',
    'hd',
    'cat',
    'amzn',
    'intc',
    't',
    'v',
    'gs'
]

In [17]:
df_raw = YahooDownloader(start_date=TRAIN_START_DATE,
                         end_date=TRADE_END_DATE,
                         ticker_list=symbols).fetch_data()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (38804, 8)


In [18]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.754725,746015200,aapl,4
1,2009-01-02,2.567500,2.726500,2.553500,2.718000,145928000,amzn,4
2,2009-01-02,44.910000,46.980000,44.709999,31.254074,7117200,cat,4
3,2009-01-02,84.019997,87.620003,82.190002,68.728455,14088500,gs,4
4,2009-01-02,23.070000,24.190001,22.959999,16.867977,14902500,hd,4


## Preprocess Data

In [19]:
fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list=INDICATORS,
                     use_vix=True,
                     use_turbulence=True,
                     user_defined_feature=False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3604, 8)
Successfully added vix
Successfully added turbulence index


In [20]:
list_ticker = processed['tic'].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(), processed['date'].max()).astype(str))
combination = list(itertools.product(list_date, list_ticker))

processed_full = pd.DataFrame(combination, columns=['date', 'tic']).merge(processed, on=['date', 'tic'], how='left')
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date', 'tic'])

processed_full = processed_full.fillna(0)

In [21]:
processed_full.tail()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
52285,2023-04-27,ibm,126.370003,127.019997,125.459999,125.261978,3204900.0,3.0,-0.810753,131.359867,122.257901,44.911367,-38.300138,1.220107,126.039049,127.767372,17.030001,6.717467
52286,2023-04-27,intc,28.750000,29.950001,28.510000,29.738222,60186200.0,3.0,-0.018936,34.073824,28.771821,50.727395,-66.298183,13.482737,30.712564,28.907981,17.030001,6.717467
52287,2023-04-27,msft,295.970001,305.200012,295.250000,304.165070,46462600.0,3.0,5.129207,298.397082,275.275828,63.072930,228.831807,42.136850,283.165982,270.070638,17.030001,6.717467
52288,2023-04-27,t,17.379999,17.629999,17.360001,17.301521,36926900.0,3.0,-0.312020,20.549149,16.734124,42.445889,-124.911611,49.531457,18.421052,18.442977,17.030001,6.717467
52289,2023-04-27,v,228.669998,229.630005,226.610001,228.564392,7128200.0,3.0,1.988247,236.272077,222.418667,54.153809,27.347515,4.300545,226.222294,224.769116,17.030001,6.717467


## Save the data

In [22]:
# Split the data
train = data_split(processed_full, TRAIN_START_DATE, TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE, TRADE_END_DATE)

print(len(train))
print(len(trade))

28930
7110


In [26]:
log_dir = "data/"
train_path = os.path.join('data', 'train_data.csv')
trade_path = os.path.join('data', 'trade_data.csv')

with open(train_path, 'w', encoding = 'utf-8-sig') as f:
    train.to_csv(f)
    
with open(trade_path, 'w', encoding = 'utf-8-sig') as f:
    trade.to_csv(f)